In [2]:
:sccache 1
// Skip this or run `cargo install sccache`

sccache: true


In [3]:
:dep z3tracer = { path = "/path/to/smt2utils/z3tracer" }
// TODO: change the path to your local path to z3tracer.

In [4]:
:dep plotters = { default_features = false, features = ["evcxr", "line_series"] }

In [5]:
:efmt {:?}

Error format: {:?} (errors must implement std::fmt::Debug)


In [6]:
:opt 3

Optimization: 3


In [7]:
use z3tracer::{syntax::*, model::*};
use std::collections::*;
use std::str::FromStr;
use plotters::prelude::*;
use plotters::evcxr::SVGWrapper;

In [8]:
:last_error_json
// BUG WORKAROUND: In case of mysteriously truncated error 'consider importing this struct', run the command above again
// to print a detailed error and find the import that is actually missing.
//
// See also :help for more useful commands

Error: 

In [9]:
/// Helper function to process files.
fn process_file(path: &str) -> std::io::Result<Model> {
    let file = std::io::BufReader::new(std::fs::File::open(path)?);
    // Inject non-default configurations here with Model::new(config).
    let mut model = Model::default();
    if let Err(le) = model.process(Some(path.to_string()), file) {
        println!("Error at {:?}: {:?}", le.position, le.error);
    }
    Ok(model)
}

/// Helper trait to print es by their id.
trait ModelExt {
    fn id2s(&self, id: &Ident) -> String;    
}

impl ModelExt for Model {
    fn id2s(&self, id: &Ident) -> String {
        self.id_to_sexp(&BTreeMap::new(), id).unwrap()
    }        
}

In [10]:
// TODO: remove after Rust issue 59278 is closed.
struct IntoIterSorted<T> {
    inner: BinaryHeap<T>,
}

impl<T> From<BinaryHeap<T>> for IntoIterSorted<T> {
    fn from(inner: BinaryHeap<T>) -> Self {
        Self { inner }
    }
}

impl<T: Ord> Iterator for IntoIterSorted<T> {
    type Item = T;

    fn next(&mut self) -> Option<T> {
        self.inner.pop()
    }

    fn size_hint(&self) -> (usize, Option<usize>) {
        let exact = self.inner.len();
        (exact, Some(exact))
    }
}

In [11]:
// Process some input file obtained with `z3 trace=true proof=true ..`
let model = process_file("../tests/data/file1.log")?;

In [12]:
// Maximum timestamp (i.e. currently, the number of lines in the Z3 log)
let max_ts = model.processed_logs();
max_ts

443166

In [13]:
// Compute top instantiated terms.
let mut top = model.most_instantiated_terms();
top.len()

66

In [14]:
// Peek at the 20th top elements
IntoIterSorted::from(top.clone()).take(20).collect::<Vec<_>>()

[(7903, basic#), (2955, datatype#85), (2955, datatype#82), (1863, datatype#), (1114, arith#), (769, datatype#59), (331, #4429!9), (304, #4328), (232, datatype#47), (224, #20151!10), (126, #20468!5), (120, #4351!1), (119, #23065!1), (119, #4347!1), (115, #4337), (100, #23092!1), (99, #23078!2), (97, datatype#53), (93, datatype#143), (93, datatype#140)]

In [15]:
// Compute top instantiated terms and retrieve the "timestamps" at which instantiations occur for each of the top terms.
let instantiation_times = IntoIterSorted::from(model.most_instantiated_terms()).map(|(_count, id)| {
    let mut timestamps = model.term_data(&id).unwrap().instantiation_timestamps.clone();
    timestamps.sort();
    let name = match model.term(&id).unwrap() { Term::Quant { name, .. } | Term::Builtin { name: Some(name) } => name, _ => "??" };
    (name.to_string(), timestamps)
}).collect::<Vec<_>>();

In [16]:
fn plot_instantiations(times: &[(String, Vec<usize>)], title: &str, top_n: usize) -> SVGWrapper {
    let max_ts = times.iter().map(|(_, v)| v.last().cloned().unwrap_or(0)).max().unwrap_or(1);
    let max_count = times[0].1.len();

    evcxr_figure((1000, 800), |root| {
        root.fill(&WHITE);
        let mut chart = ChartBuilder::on(&root)
            .caption(title, ("Arial", 30).into_font())
            .margin(10)
            .x_label_area_size(40)
            .y_label_area_size(50)
            .build_cartesian_2d(0..max_ts, 0..max_count)?;

        chart.configure_mesh().y_desc("Cumulative number of instantiations").x_desc("Time (line number)").draw()?;

        for (j, (name, values)) in times.iter().take(top_n).enumerate() {
            let color : PaletteColor<Palette9999> = PaletteColor::pick(j);
            chart
            .draw_series(
                LineSeries::new(values.iter().enumerate().map(|(i, c)| (*c, i)), &color)
            ).unwrap()
            .label(name)
            .legend(move |(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], color.filled()));
        }

        chart.configure_series_labels()
            .background_style(&WHITE.mix(0.8))
            .border_style(&BLACK)
            .position(SeriesLabelPosition::UpperLeft)
            .draw()?;
        Ok(())
    })
}

In [17]:
plot_instantiations(&instantiation_times, "Top 10 Quantifiers Instantiations", 10)

Top 10 Quantifiers Instantiations
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Cumulative number of instantiations
 
 
Time (line number)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
1000
 
 
 
2000
 
 
 
3000
 
 
 
4000
 
 
 
5000
 
 
 
6000
 
 
 
7000
 
 
 
 
0
 
 
 
50000
 
 
 
100000
 
 
 
150000
 
 
 
200000
 
 
 
250000
 
 
 
300000
 
 
 
350000
 
 
 
400000
 
 
<polyline fill="none" opacity="1" stroke="#FFE119" stroke-width="1" points="60,749 60,748 60,748 60,748 60,748 61,748 61,748 61,748 61,748 61,748 61,748 61,748 61,747 61,747 62,747 62,747 62,747 62,747 62,747 62,747 64,747 64,747 64,747 64,746 64,746 64,746 64,746 64,746 64,746 64,746 65,746 65,746 65,746 65,746 65,745 65,745 65,745 65,745 65,745 65,745 65,745 65,745 65,745 65,745 65,745 65,744 65,744 67,744 67,744 68,744 68,744 68,744 68,744 68,744 68,744 68,744 70,744 70,743 70,743 70,743 81,743 82,743 82,743 82,743 82,743 82,743 85,743 85,743 86,742 86,742 86,742 86,742 86,742 86,742 86,742 86,742 86,742 86,742 86,742 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,741 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,740 86,739 86,739 86,739 87,739 87,739 87,739 87,739 87,739 87,739 87,739 87,739 87,739 87,738 87,738 87,738 87,738 87,738 87,738 87,738 87,738 87,738 87,738 87,738 88,737 88,737 88,737 88,737 88,737 88,737 88,737 88,737 88,737 88,737 88,737 89,736 89,736 89,736 89,736 89,736 94,736 94,736 94,736 94,736 94,736 94,736 95,735 95,735 95,735 95,735 95,735 95,735 95,735 95,735 95,735 95,735 95,735 95,734 95,734 95,734 95,734 95,734 95,734 95,734 95,734 95,734 95,734 95,734 95,734 95,733 95,733 95,733 95,733 95,733 95,733 96,733 96,733 96,733 96,733 96,733 96,732 96,732 97,732 97,732 97,732 97,732 97,732 97,732 97,732 97,732 97,732 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,731 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,730 97,729 98,729 98,729 98,729 98,729 98,729 98,729 98,729 142,729 142,729 142,729 142,729 142,728 142,728 142,728 142,728 142,728 142,728 142,728 142,728 142,728 143,728 143,728 143,727 143,727 144,727 144,727 144,727 144,727 144,727 144,727 144,727 144,727 144,727 144,726 144,726 144,726 144,726 144,726 145,726 145,726 145,726 145,726 145,726 145,726 145,725 145,725 145,725 145,725 145,725 145,725 146,725 146,725 146,725 146,725 146,725 146,725 146,724 146,724 146,724 146,724 146,724 146,724 146,724 147,724 147,724 147,724 147,724 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 147,723 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,722 148,721 148,721 148,721 148,721 149,721 149,721 149,721 149,721 149,721 149,721 149,721 149,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,720 150,719 150,719 150,719 151,719 151,719 151,719 151,719 151,719 151,719 151,719 151,719 151,718 151,718 151,718 151,718 151,718 151,718 151,718 151,718 151,718 152,718 152,718 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,717 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 152,716 153,716 153,715 153,715 153,715 153,715 153,715 153,715 153,715 153,715 153,715 154,715 154,715 154,715 154,714 154,714 154,714 154,714 154,714 154,714 154,714 154,714 155,714 155,714 155,714 155,713 155,713 155,713 155,713 155,713 155,713 155,713 155,713 156,713 156,713 156,713 156,712 156,712 156,712 156,712 156,712 156,712 157,712 157,712 157,712 157,712 157,712 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 157,711 158,711 158,710 158,710 158,710 158,710 158,710 158,710 158,710 158,710 15

In [18]:
// Top instantiated quantified term.
model.id2s(&top.peek()?.1)

"basic"

In [19]:
// Final counter and `Ident` value for the top term.
top.peek()?

(7903, basic#)

In [20]:
// Move to the first user quantifier
while !model.term(&top.peek()?.1)?.name().unwrap_or("").starts_with("outputbpl") { top.pop(); }
top.peek()?

(331, #4429!9)

In [21]:
// Introspect a particular term, given by handle.
// Note that the version number `!9` was added during parsing for disambiguation and is not present in the original log file.
model.term(&Ident::from_str("#4429!9")?)?

Quant { name: "outputbpl.310:31", params: 2, triggers: [#284], body: #4443, var_names: Some([VarName { name: Symbol("v2"), sort: Symbol("T@$Value") }, VarName { name: Symbol("v1"), sort: Symbol("T@$Value") }]) }

In [22]:
model.term_data(&Ident::from_str("#4429!9")?)?.instantiations.len()

359

In [23]:
// Create a new chart only with quantifier instantiation inside the file `outputbpl`.
let user_instantiation_times = instantiation_times.clone().into_iter().filter(|(n, _)| n.starts_with("outputbpl")).collect::<Vec<_>>();

plot_instantiations(&user_instantiation_times, "Top 10 User Quantifiers Instantiations", 10)

Top 10 User Quantifiers Instantiations
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Cumulative number of instantiations
 
 
Time (line number)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
 
0
 
 
 
50000
 
 
 
100000
 
 
 
150000
 
 
 
200000
 
 
 
250000
 
 
 
300000
 
 
 
350000
 
 
 
400000
 
 
<polyline fill="none" opacity="1" stroke="#FFE119" stroke-width="1" points="255,749 256,746 257,744 291,742 292,740 293,738 296,736 296,734 297,731 301,729 302,727 305,725 307,723 308,721 308,719 309,717 341,714 341,712 342,710 346,708 347,706 347,704 348,702 351,700 352,697 386,695 386,693 387,691 391,689 394,687 394,685 394,682 395,680 396,678 398,676 398,674 398,672 399,670 400,668 400,665 401,663 402,661 403,659 403,657 404,655 404,653 405,651 405,648 406,646 406,644 448,642 448,640 448,638 449,636 449,633 450,631 450,629 451,627 452,625 452,623 452,621 453,619 454,616 454,614 455,612 455,610 455,608 456,606 456,604 457,602 457,599 457,597 458,595 460,593 460,591 461,589 461,587 462,584 462,582 480,580 481,578 481,576 482,574 482,572 483,570 483,567 490,565 490,563 491,561 511,559 512,557 513,555 514,553 516,550 517,548 533,546 534,544 535,542 536,540 536,538 547,536 548,533 551,531 551,529 554,527 554,525 556,523 557,521 557,518 569,516 573,514 574,512 574,510 584,508 584,506 585,504 585,501 586,499 586,497 586,495 587,493 587,491 588,489 594,487 594,484 612,482 613,480 613,478 615,476 617,474 617,472 618,469 619,467 620,465 620,463 624,461 626,459 626,457 627,455 632,452 632,450 633,448 633,446 635,444 635,442 638,440 639,438 652,435 653,433 655,431 655,429 655,427 656,425 656,423 656,420 657,418 658,416 658,414 659,412 659,410 659,408 660,406 660,403 660,401 660,399 661,397 661,395 661,393 664,391 664,389 665,386 665,384 666,382 666,380 666,378 667,376 667,374 667,372 668,369 668,367 669,365 669,363 686,361 687,359 687,357 688,354 688,352 688,350 689,348 689,346 689,344 689,342 694,340 694,337 694,335 695,333 695,331 695,329 712,327 713,325 714,323 715,320 715,318 716,316 716,314 719,312 723,310 723,308 724,305 724,303 724,301 725,299 725,297 726,295 726,293 727,291 728,288 728,286 728,284 729,282 729,280 730,278 730,276 730,274 740,271 741,269 741,267 741,265 742,263 744,261 744,259 744,256 744,254 745,252 745,250 760,248 760,246 760,244 761,242 762,239 762,237 762,235 763,233 763,231 763,229 772,227 773,225 779,222 779,220 779,218 779,216 780,214 780,212 789,210 789,208 795,205 795,203 796,201 796,199 796,197 797,195 797,193 798,190 800,188 800,186 801,184 801,182 801,180 802,178 802,176 819,173 820,171 820,169 820,167 822,165 822,163 842,161 842,159 843,156 847,154 847,152 849,150 849,148 850,146 850,144 850,141 851,139 871,137 871,135 872,133 872,131 872,129 873,127 873,124 894,122 898,120 902,118 902,116 903,114 903,112 903,110 921,107 921,105 922,103 922,101 923,99 924,97 925,95 926,92 942,90 950,88 951,86 951,84 951,82 952,80 952,78 965,75 966,73 967,71 969,69 969,67 970,65 970,63 971,61 971,58 971,56 971,54 972,52 972,50 984,48 984,46 "/>
<polyline fill="none" opacity="1" stroke="#0082C8" stroke-width="1" points="255,749 256,746 256,744 290,742 291,740 292,738 295,736 296,734 297,731 301,729 305,727 307,725 307,723 308,721 309,719 340,717 341,714 341,712 346,710 346,708 347,706 347,704 350,702 351,700 352,697 385,695 386,693 387,691 389,689 391,687 393,685 394,682 394,680 394,678 395,676 397,674 398,672 398,670 398,668 399,665 400,663 400,661 400,659 401,657 402,655 403,653 403,651 404,648 404,646 404,644 406,642 447,640 448,638 448,636 449,633 450,631 450,629 451,627 451,625 452,623 452,621 452,619 453,616 454,614 455,612 455,610 455,608 456,606 457,604 457,602 457,599 459,597 460,595 460,593 